# Clustering K-Prototypes pour Segmentation Client

## Objectif
Utiliser l'algorithme K-Prototypes pour découvrir des clusters naturels dans les données mixtes (numériques + catégorielles) et évaluer leur correspondance avec les segments réels (A/B/C/D).

## Méthode : K-Prototypes
K-Prototypes est une extension de K-Means adaptée aux données mixtes :
- Utilise la distance euclidienne pour les variables numériques (Age, Work_Experience, Family_Size)
- Utilise le matching simple pour les variables catégorielles (Gender, Profession, etc.)
- Combine les deux distances avec un facteur gamma pour pondérer l'importance relative

Cette approche évite de devoir encoder arbitrairement les variables catégorielles, contrairement à K-Means classique.

## Hypothèse
Les clusters naturels découverts par K-Prototypes peuvent révéler une structure latente utile pour améliorer la prédiction des segments, particulièrement pour distinguer B et C qui se chevauchent.


In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score, silhouette_score

import warnings
warnings.filterwarnings('ignore')



In [2]:
%pip install -q kmodes
from kmodes.kprototypes import KPrototypes


Note: you may need to restart the kernel to use updated packages.


## 1. Chargement et Préparation des Données


In [3]:
train = pd.read_csv('../Data/Train.csv')
test = pd.read_csv('../Data/Test.csv')

print(f"Train shape: {train.shape}")
print(f"Test shape: {test.shape}")
print(f"\nMissing values (Train):")
print(train.isnull().sum())


Train shape: (8068, 11)
Test shape: (2627, 10)

Missing values (Train):
ID                   0
Gender               0
Ever_Married       140
Age                  0
Graduated           78
Profession         124
Work_Experience    829
Spending_Score       0
Family_Size        335
Var_1               76
Segmentation         0
dtype: int64


In [4]:
# K-Prototypes ne supporte pas les valeurs manquantes
# Stratégie d'imputation simple pour le clustering exploratoire

train_clean = train.copy()
test_clean = test.copy()

# Variables catégorielles : imputation par une modalité "Unknown"
# Permet de conserver l'information de missingness qui peut être informative
train_clean['Ever_Married'] = train_clean['Ever_Married'].fillna('No')
train_clean['Graduated'] = train_clean['Graduated'].fillna('No')
train_clean['Profession'] = train_clean['Profession'].fillna('Unknown')
train_clean['Var_1'] = train_clean['Var_1'].fillna('Unknown')

# Variables numériques : imputation par la médiane (robuste aux outliers)
train_clean['Work_Experience'] = train_clean['Work_Experience'].fillna(train_clean['Work_Experience'].median())
train_clean['Family_Size'] = train_clean['Family_Size'].fillna(train_clean['Family_Size'].median())

# Même traitement sur le test
test_clean['Ever_Married'] = test_clean['Ever_Married'].fillna('No')
test_clean['Graduated'] = test_clean['Graduated'].fillna('No')
test_clean['Profession'] = test_clean['Profession'].fillna('Unknown')
test_clean['Var_1'] = test_clean['Var_1'].fillna('Unknown')
test_clean['Work_Experience'] = test_clean['Work_Experience'].fillna(test_clean['Work_Experience'].median())
test_clean['Family_Size'] = test_clean['Family_Size'].fillna(test_clean['Family_Size'].median())


## 2. Clustering K-Prototypes


In [5]:
# Séparer variables numériques et catégorielles
# K-Prototypes calcule des distances différentes pour chaque type
categorical_cols = ['Gender', 'Ever_Married', 'Graduated', 'Profession', 'Spending_Score', 'Var_1']
numeric_cols = ['Age', 'Work_Experience', 'Family_Size']

# Préparation du format attendu par K-Prototypes
# L'algorithme attend un array avec les numériques en premier, puis les catégorielles
cat_data_train = train_clean[categorical_cols].astype(str)
num_data_train = train_clean[numeric_cols].astype(float)

# Concaténation horizontale : [num1, num2, num3, cat1, cat2, ...]
combo_data_train = np.hstack([num_data_train.values, cat_data_train.values])

# Indices des colonnes catégorielles après concaténation
# Nécessaire pour indiquer à K-Prototypes où appliquer le matching
cat_indices = list(range(len(numeric_cols), combo_data_train.shape[1]))

print(f"Données combinées : {combo_data_train.shape}")
print(f"Indices catégoriels : {cat_indices}")


Données combinées : (8068, 9)
Indices catégoriels : [3, 4, 5, 6, 7, 8]


In [6]:
# Fit K-Prototypes avec 4 clusters (correspondant à A, B, C, D)
kproto = KPrototypes(n_clusters=4, init='Huang', random_state=42, verbose=1)
clusters_train = kproto.fit_predict(combo_data_train, categorical=cat_indices)

train_clean['KProto_Cluster'] = clusters_train

print(f"\n✅ K-Prototypes clustering terminé")
print(f"Distribution des clusters:")
print(train_clean['KProto_Cluster'].value_counts().sort_index())


Init: initializing centroids


Init: initializing clusters
Starting iterations...
Run: 1, iteration: 1/100, moves: 2180, ncost: 408030.8351452565
Run: 1, iteration: 2/100, moves: 892, ncost: 375730.2543576146
Run: 1, iteration: 3/100, moves: 668, ncost: 357065.95979767665
Run: 1, iteration: 4/100, moves: 332, ncost: 352387.12293379725
Run: 1, iteration: 5/100, moves: 262, ncost: 350435.7782383151
Run: 1, iteration: 6/100, moves: 108, ncost: 350139.90846137115
Run: 1, iteration: 7/100, moves: 74, ncost: 349968.4516105981
Run: 1, iteration: 8/100, moves: 85, ncost: 349708.90808115137
Run: 1, iteration: 9/100, moves: 67, ncost: 349565.71024181374
Run: 1, iteration: 10/100, moves: 60, ncost: 349461.2122888842
Run: 1, iteration: 11/100, moves: 28, ncost: 349436.89701392205
Run: 1, iteration: 12/100, moves: 12, ncost: 349432.3206235398
Run: 1, iteration: 13/100, moves: 0, ncost: 349432.3206235398
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run: 2, iteration: 1/100, moves: 1810, ncost: 6

## 3. Analyse de la Correspondance Clusters vs Segments Réels


In [7]:
# Encoder les segments (A=0, B=1, C=2, D=3)
seg_encoder = LabelEncoder()
train_clean['Segmentation_encoded'] = seg_encoder.fit_transform(train_clean['Segmentation'])

print("Mapping des segments:")
for i, label in enumerate(seg_encoder.classes_):
    print(f"{label} -> {i}")


Mapping des segments:
A -> 0
B -> 1
C -> 2
D -> 3


In [8]:
# Table de contingence : clusters découverts vs segments réels
# Permet de voir si les clusters naturels correspondent aux segments business
crosstab = pd.crosstab(train_clean['KProto_Cluster'], train_clean['Segmentation'], margins=True)

print("\nCorrespondance Clusters K-Prototypes vs Segments Réels:")
print(crosstab)



Correspondance Clusters K-Prototypes vs Segments Réels:
Segmentation       A     B     C     D   All
KProto_Cluster                              
0                304   293   302   180  1079
1                382   607   795   151  1935
2                773   684   576   385  2418
3                513   274   297  1552  2636
All             1972  1858  1970  2268  8068


In [9]:
# Heatmap normalisée : proportion de chaque segment dans chaque cluster
# Normalization='index' : chaque ligne (cluster) somme à 100%
crosstab_norm = pd.crosstab(train_clean['KProto_Cluster'], train_clean['Segmentation'], normalize='index')

# Conversion en pourcentage pour l'affichage
crosstab_pct = crosstab_norm * 100

fig = go.Figure(data=go.Heatmap(
    z=crosstab_pct.values,
    x=crosstab_pct.columns,
    y=[f'Cluster {i}' for i in crosstab_pct.index],
    text=crosstab_pct.values.round(1),
    texttemplate='%{text}%',
    textfont={"size": 12},
    colorscale='YlOrRd',
    colorbar=dict(title="Proportion (%)")
))

fig.update_layout(
    title='Correspondance Clusters K-Prototypes vs Segments Réels',
    xaxis_title='Segment Réel',
    yaxis_title='Cluster K-Prototypes',
    height=500,
    width=700
)

fig.show()


In [10]:
# Métriques de qualité du clustering
# ARI et NMI mesurent la correspondance entre clusters et segments réels

# Adjusted Rand Index : mesure la similarité entre deux partitions
# Ajusté pour le hasard (0 = partitions aléatoires, 1 = partitions identiques)
ari = adjusted_rand_score(train_clean['Segmentation_encoded'], train_clean['KProto_Cluster'])

# Normalized Mutual Information : information mutuelle normalisée
# Mesure l'information partagée entre les deux partitions (0 = indépendant, 1 = identique)
nmi = normalized_mutual_info_score(train_clean['Segmentation_encoded'], train_clean['KProto_Cluster'])

# Silhouette Score : mesure la cohésion intra-cluster et séparation inter-cluster
# Nécessite un encodage des catégorielles pour le calcul de distance euclidienne
train_encoded = train_clean.copy()
for col in categorical_cols:
    le = LabelEncoder()
    train_encoded[col] = le.fit_transform(train_encoded[col])

X_silhouette = train_encoded[numeric_cols + categorical_cols]
silhouette = silhouette_score(X_silhouette, train_clean['KProto_Cluster'])

print("\nMétriques de qualité du clustering:")
print(f"Adjusted Rand Index (ARI) : {ari:.4f}")
print(f"  -> Mesure la similarité avec les segments réels")
print(f"\nNormalized Mutual Information (NMI) : {nmi:.4f}")
print(f"  -> Mesure l'information partagée avec les segments")
print(f"\nSilhouette Score : {silhouette:.4f}")
print(f"  -> Mesure la qualité intrinsèque des clusters")


Métriques de qualité du clustering:
Adjusted Rand Index (ARI) : 0.1110
  -> Mesure la similarité avec les segments réels

Normalized Mutual Information (NMI) : 0.0968
  -> Mesure l'information partagée avec les segments

Silhouette Score : 0.3987
  -> Mesure la qualité intrinsèque des clusters

Interprétation : Faible correspondance - structure différente


## 4. Visualisation t-SNE


In [11]:
# t-SNE (t-Distributed Stochastic Neighbor Embedding)
# Algorithme de réduction dimensionnelle non-linéaire
# Préserve les distances locales entre points (voisinage) lors de la projection 2D
# Utile pour visualiser la structure des clusters dans un espace réduit

X_tsne = train_encoded[numeric_cols + categorical_cols].values

# Paramètres :
# - perplexity=30 : balance entre structure locale et globale (5-50 généralement)
# - max_iter=1000 : nombre d'itérations pour l'optimisation
print("Calcul t-SNE en cours (1-2 minutes)...")

tsne = TSNE(n_components=2, random_state=42, perplexity=30, max_iter=1000, verbose=1)
tsne_result = tsne.fit_transform(X_tsne)

# Ajout des coordonnées t-SNE au dataframe
train_clean['tsne_1'] = tsne_result[:, 0]
train_clean['tsne_2'] = tsne_result[:, 1]


Calcul t-SNE en cours (1-2 minutes)...
[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 8068 samples in 0.032s...
[t-SNE] Computed neighbors for 8068 samples in 0.452s...
[t-SNE] Computed conditional probabilities for sample 1000 / 8068
[t-SNE] Computed conditional probabilities for sample 2000 / 8068
[t-SNE] Computed conditional probabilities for sample 3000 / 8068
[t-SNE] Computed conditional probabilities for sample 4000 / 8068
[t-SNE] Computed conditional probabilities for sample 5000 / 8068
[t-SNE] Computed conditional probabilities for sample 6000 / 8068
[t-SNE] Computed conditional probabilities for sample 7000 / 8068
[t-SNE] Computed conditional probabilities for sample 8000 / 8068
[t-SNE] Computed conditional probabilities for sample 8068 / 8068
[t-SNE] Mean sigma: 1.215552
[t-SNE] KL divergence after 250 iterations with early exaggeration: 74.164925
[t-SNE] KL divergence after 1000 iterations: 1.094037


In [12]:
# Visualisation comparative : Clusters découverts vs Segments réels
# Permet d'observer visuellement la correspondance entre les deux partitions

from plotly.subplots import make_subplots

fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Clusters K-Prototypes (découverts)', 'Segments Réels (A, B, C, D)'),
    horizontal_spacing=0.1
)

# Subplot 1 : Clusters K-Prototypes
fig.add_trace(
    go.Scatter(
        x=train_clean['tsne_1'],
        y=train_clean['tsne_2'],
        mode='markers',
        marker=dict(
            color=train_clean['KProto_Cluster'],
            colorscale='Viridis',
            size=5,
            opacity=0.6,
            colorbar=dict(title="Cluster", x=0.45)
        ),
        showlegend=False
    ),
    row=1, col=1
)

# Subplot 2 : Segments réels  
segment_colors = {0: 'red', 1: 'blue', 2: 'green', 3: 'orange'}
segment_names = {0: 'A', 1: 'B', 2: 'C', 3: 'D'}

for seg_id in sorted(train_clean['Segmentation_encoded'].unique()):
    mask = train_clean['Segmentation_encoded'] == seg_id
    fig.add_trace(
        go.Scatter(
            x=train_clean.loc[mask, 'tsne_1'],
            y=train_clean.loc[mask, 'tsne_2'],
            mode='markers',
            marker=dict(
                color=segment_colors[seg_id],
                size=5,
                opacity=0.6
            ),
            name=f'Segment {segment_names[seg_id]}',
            showlegend=True
        ),
        row=1, col=2
    )

fig.update_xaxes(title_text="t-SNE Dimension 1", row=1, col=1)
fig.update_xaxes(title_text="t-SNE Dimension 1", row=1, col=2)
fig.update_yaxes(title_text="t-SNE Dimension 2", row=1, col=1)
fig.update_yaxes(title_text="t-SNE Dimension 2", row=1, col=2)

fig.update_layout(
    title_text="Projection t-SNE : Clusters vs Segments Réels",
    height=600,
    width=1400,
    showlegend=True
)

fig.show()


## 5. Conclusion

Résultats décevants. Le clustering ne capture pas la structure des segments réels.

Matrice de contingence :
- Cluster 0 : ~28% de A, B et C chacun, moins de D
- Cluster 1 : un peu plus de C (41%) mais beaucoup de B aussi (31%)  
- Cluster 2 : encore une distribution quasi-uniforme
- Cluster 3 : le seul un peu propre avec 59% de D, mais ça reste moyen

ARI=0.11 et NMI=0.10, c'est vraiment faible. Presque du hasard. Le Silhouette à 0.40 montre une certaine cohésion interne des clusters, mais ça ne correspond pas aux segments business.

Pourquoi ça marche pas ?

Soit la segmentation repose sur des règles métier spécifiques (seuils, combinaisons) que le clustering par distance ne peut pas capturer. Soit B et C se chevauchent trop dans l'espace des features (déjà observé dans l'EDA). Soit les variables disponibles sont insuffisantes.

Bref, utiliser les clusters comme features additionnelles ne servira pas à grand chose. Trop de bruit, pas assez de signal. On reste sur du supervisé classique (LGBM) qui peut apprendre les frontières de décision complexes directement. 